In [1]:
# ! pip install nvidia-tensorrt==8.4.3.1

In [2]:
%cd ../

/media/iec-ptit/b3a4e3e5-118e-48d7-ae64-40d0d4dae3a6/home/iec-ptit/Desktop/vvinh/vsmoked


In [3]:
from dataset.dataset import SmokeDetectionDataset
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm



smkd_dataset = SmokeDetectionDataset('./dataset/wildfire_smoke_dataset/classification/test','./dataset/wildfire_smoke_dataset/test/labels',sequence_length=7)
test_loader = DataLoader(dataset=smkd_dataset, batch_size=1, pin_memory=True)

2024-04-21 20:26:59.543674: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-21 20:26:59.572132: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-21 20:27:00.052277: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


['nonsmoke', 'smoke']


load smoke images to cache: 100%|██████████| 10/10 [00:13<00:00,  1.39s/it]


In [26]:
def dict_of_label(y):
    return [{
            'label': yy[0].item(),
            'box': [yy[1][0].item(), 
                    yy[1][1].item(), 
                    yy[1][2].item(), 
                    yy[1][3].item()],
            'conf': 1.0
            } for yy in y]


def iou(box1, box2):
        # Extract coordinates of the boxes
    xmin1, ymin1, xmax1, ymax1 = box1
    xmin2, ymin2, xmax2, ymax2 = box2
    
    # Calculate the intersection coordinates
    x_left = max(xmin1, xmin2)
    y_top = max(ymin1, ymin2)
    x_right = min(xmax1, xmax2)
    y_bottom = min(ymax1, ymax2)
    
    # Calculate the intersection area
    intersection_area = max(0, x_right - x_left) * max(0, y_bottom - y_top)
    
    # Calculate the areas of the bounding boxes
    box1_area = (xmax1 - xmin1) * (ymax1 - ymin1)
    box2_area = (xmax2 - xmin2) * (ymax2 - ymin2)
    
    # Calculate the union area
    union_area = box1_area + box2_area - intersection_area
    
    # Calculate IoU
    iou = intersection_area / union_area if union_area > 0 else 0
    
    return iou

def calculate_detection_accuracy(predicted_boxes, gt_boxes, iou_threshold=0.3):
    """
    input: predicted_boxes, gt_boxes, iou_threshold
    return tp, tn, fp, fn
    """
    total_gt_objects = len(gt_boxes)
    total_predicted_objects = len(predicted_boxes)
    tp = tn = fp = fn = 0
    if total_gt_objects==0 & total_predicted_objects==0:
        tn = 1
        return tp, tn, fp, fn
    if total_gt_objects==0 & total_predicted_objects!=0:
        fp = total_predicted_objects
        return tp, tn, fp, fn
    if total_gt_objects!=0 & total_predicted_objects==0:
        fn = total_gt_objects

    match_matrix = np.zeros((total_predicted_objects,total_gt_objects))
    for pred_idx in range(total_predicted_objects):
        for gt_idx in range(total_gt_objects): 
            if iou(predicted_boxes[pred_idx],gt_boxes[gt_idx]) > iou_threshold:
                match_matrix[pred_idx][gt_idx] = 1
    gt_sum = np.sum(match_matrix, 0)
    tp = np.sum(gt_sum>0)
    fn = np.sum(gt_sum==0)
    fp = np.sum(np.sum(match_matrix,1)==0)
    return tp, tn, fp, fn



#  print(f'accuracy: {round(num_correct / num_samples, 4)}')

In [27]:
from vsmoked.object_detection import Detector

In [28]:
# !trtexec --onnx=model/OurModel/model_18_04/best_ds_nms.trt --uint8 --avgRuns=100 --duration=15

In [39]:
detector = Detector('./vsmoked/models/best_ds_nms.trt','./vsmoked/models/sky_seg.trt',use_SGS_block=True)
# detector.set_params(7,2, 0.7, 0.014, 0.014)
detector.set_params(7,2, 0.7, -1, -1)

reading  ./vsmoked/models/best_ds_nms.trt
./vsmoked/models/sky_seg.onnx
reading  ./vsmoked/models/sky_seg.trt


In [40]:
conf_threshold = 0.15
iou_threshold = 0.3
import time
# predicted_boxes
loop = tqdm(test_loader, total=len(test_loader), position=0, leave=True)
total_tp = total_tn = total_fp = total_fn = 0
epsilon = 0.0000001
min_mm = 1
latency=[]
for x, y in test_loader:
    
    x = np.array([i.numpy().squeeze(0) for i in x])
    if(len(y)!=0):
        y = dict_of_label(y)
    else:
        continue
    time,preds = detector.detect(x,conf_threshold)
    latency.append(time)
    gt_boxes = list(map(lambda yy: yy['box'],y))
    pred_boxes = list(map(lambda yy: yy['box'],preds))
    tp, tn, fp, fn = calculate_detection_accuracy(pred_boxes, gt_boxes,iou_threshold)
    if fn !=0:
        print(f'\npreds:{preds})\ntrust:{y}')
    total_tp+=tp
    total_tn+=tn
    total_fp+=fp
    total_fn+=fn
    # print(images[0],preds,y)
    # time.sleep(1)
    precision = total_tp/(total_tp+total_fp+epsilon)
    recall = total_tp/(total_tp+total_fn+epsilon)
    f1 = 2*precision*recall/(precision+recall+epsilon)
    acc = (total_tp+total_tn)/(total_tp+total_tn+total_fn+total_fp)
    loop.set_postfix(precision=f'{precision:.4f}', recall=f'{recall:.4f}', f1=f'{f1:.4f}', acc=f'{acc:.4f}')
    loop.update(1)

  2%|▏         | 68/2911 [00:12<07:26,  6.37it/s, acc=0.8272, f1=0.9054, precision=0.8375, recall=0.9853] 

motionless mmb

preds:[{'label': 0.0, 'box': [0.1015625, 0.0234375, 0.12656250596046448, 0.0703125], 'conf': 0.20214844, 'mm': 0.08480664, 'mmb': 0.18495914}])
trust:[{'label': 0, 'box': [0.1047794117647059, 0.00392156862745098, 0.16875, 0.07254901960784314], 'conf': 1.0}]


  8%|▊         | 241/2911 [00:36<06:17,  7.08it/s, acc=0.8511, f1=0.9195, precision=0.8541, recall=0.9959]

motionless mm


  8%|▊         | 243/2911 [00:36<06:31,  6.81it/s, acc=0.8521, f1=0.9202, precision=0.8551, recall=0.9959]

...

 32%|███▏      | 937/2911 [02:14<04:05,  8.03it/s, acc=0.8129, f1=0.8968, precision=0.8245, recall=0.9829]


preds:[{'label': 0.0, 'box': [0.33906251192092896, 0.08906249701976776, 0.40312498807907104, 0.22187499701976776], 'conf': 0.8413086, 'mm': 0.039039984, 'mmb': 0.035504036}])
trust:[{'label': 0, 'box': [0.35859375, 0.16805555555555557, 0.40546875, 0.2236111111111111], 'conf': 1.0}]


 33%|███▎      | 947/2911 [02:16<04:51,  6.74it/s, acc=0.8103, f1=0.8952, precision=0.8217, recall=0.9831]

motionless mm


 35%|███▌      | 1030/2911 [02:27<04:29,  6.98it/s, acc=0.8191, f1=0.9005, precision=0.8298, recall=0.9845]

In [9]:
np.mean(latency)

0.014040004628375896

### Measure Object detector model latency 

In [10]:
import time
def timeit(fn): 
    # *args and **kwargs are to support positional and named arguments of fn
    def get_time(*args, **kwargs): 
        start = time.time() 
        output = fn(*args, **kwargs)
        print(f"Time taken in {fn.__name__}: {time.time() - start:.7f}")
        return output  # make sure that the decorator returns the output of fn
    return get_time 


In [11]:
detect_raw = detector.detector
# input = np.random.randint(0, 256, size=(1,3,640,640),dtype='int8')
input = np.random.randint(low=0, high=256, size=(1, 3, 640, 640), dtype=np.uint8)

In [12]:
@timeit
def measure_object_detector_latency(input):
    for i in range(1000):
        detect_raw.detect_raw(input)

In [13]:
# warm up
for i in range(500):
    detect_raw.detect_raw(input)
measure_object_detector_latency(input)
  


Time taken in measure_object_detector_latency: 12.0210667


### Measure Motion cost

In [14]:
from model.OurModel.object_detection import motion_measure
import numpy as np
import cv2
import time

ModuleNotFoundError: No module named 'model'

In [ ]:
@timeit
def motion_measure_latency(input):
    for i in range(1000):
        motion_measure(input)
    

In [ ]:
# # Generate random RGB image data
# image_list = [np.random.randint(0, 256, (75, 75, 3), dtype=np.uint8) for i in range(2)]
# for i in range(1000):
#     motion_measure_latency(image_list)

In [ ]:
# Generate random RGB image data
image_list = [np.random.randint(0, 256, (75, 75, 3), dtype=np.uint8) for i in range(3)]
for i in range(1000):
    motion_measure_latency(image_list)

Time taken in motion_measure_latency: 0.0366106
Time taken in motion_measure_latency: 0.0251179
Time taken in motion_measure_latency: 0.0261419
Time taken in motion_measure_latency: 0.0257306
Time taken in motion_measure_latency: 0.0266714
Time taken in motion_measure_latency: 0.0249796
Time taken in motion_measure_latency: 0.0249269
Time taken in motion_measure_latency: 0.0270162
Time taken in motion_measure_latency: 0.0251889
Time taken in motion_measure_latency: 0.0249763
Time taken in motion_measure_latency: 0.0248358
Time taken in motion_measure_latency: 0.0263648
Time taken in motion_measure_latency: 0.0250151
Time taken in motion_measure_latency: 0.0249858
Time taken in motion_measure_latency: 0.0248122
Time taken in motion_measure_latency: 0.0263348
Time taken in motion_measure_latency: 0.0249798
Time taken in motion_measure_latency: 0.0248449
Time taken in motion_measure_latency: 0.0246298
Time taken in motion_measure_latency: 0.0269072
Time taken in motion_measure_latency: 0.

100%|██████████| 2911/2911 [03:08<00:00, 16.29it/s, acc=0.7826, f1=0.7452, precision=0.6155, recall=0.9442]

Time taken in motion_measure_latency: 0.0258298
Time taken in motion_measure_latency: 0.0255687
Time taken in motion_measure_latency: 0.0248439
Time taken in motion_measure_latency: 0.0252075


KeyboardInterrupt: 

In [ ]:
# Generate random RGB image data
image_list = [np.random.randint(0, 256, (75, 75, 3), dtype=np.uint8) for i in range(4)]
for i in range(1000):
    motion_measure_latency(image_list)

Time taken in motion_measure_latency: 0.0384216
Time taken in motion_measure_latency: 0.0377262
Time taken in motion_measure_latency: 0.0380383
Time taken in motion_measure_latency: 0.0379813
Time taken in motion_measure_latency: 0.0386267
Time taken in motion_measure_latency: 0.0378759
Time taken in motion_measure_latency: 0.0387757
Time taken in motion_measure_latency: 0.0381277
Time taken in motion_measure_latency: 0.0382090
Time taken in motion_measure_latency: 0.0384436
Time taken in motion_measure_latency: 0.0383945
Time taken in motion_measure_latency: 0.0383041
Time taken in motion_measure_latency: 0.0380559
Time taken in motion_measure_latency: 0.0383222
Time taken in motion_measure_latency: 0.0383422
Time taken in motion_measure_latency: 0.0378745
Time taken in motion_measure_latency: 0.0386131
Time taken in motion_measure_latency: 0.0384061
Time taken in motion_measure_latency: 0.0383461
Time taken in motion_measure_latency: 0.0385673
Time taken in motion_measure_latency: 0.

In [ ]:
# Generate random RGB image data
image_list = [np.random.randint(0, 256, (75, 75, 3), dtype=np.uint8) for i in range(5)]
for i in range(1000):
    motion_measure_latency(image_list)

Time taken in motion_measure_latency: 0.0500073
Time taken in motion_measure_latency: 0.0501926
Time taken in motion_measure_latency: 0.0493708
Time taken in motion_measure_latency: 0.0490484
Time taken in motion_measure_latency: 0.0492318
Time taken in motion_measure_latency: 0.0497720
Time taken in motion_measure_latency: 0.0498037
Time taken in motion_measure_latency: 0.0495512
Time taken in motion_measure_latency: 0.0492113
Time taken in motion_measure_latency: 0.0491459
Time taken in motion_measure_latency: 0.0498028
Time taken in motion_measure_latency: 0.0497510
Time taken in motion_measure_latency: 0.0494258
Time taken in motion_measure_latency: 0.0493698
Time taken in motion_measure_latency: 0.0492198
Time taken in motion_measure_latency: 0.0495706
Time taken in motion_measure_latency: 0.0495234
Time taken in motion_measure_latency: 0.0491002
Time taken in motion_measure_latency: 0.0491047
Time taken in motion_measure_latency: 0.0490704
Time taken in motion_measure_latency: 0.